In [ ]:
import base64
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
from Crypto import Random

def encrypt(key, source, encode=True):
    key = SHA256.new(key).digest()  # use SHA-256 over our key to get a proper-sized AES key
    IV = Random.new().read(AES.block_size)  # generate IV
    encryptor = AES.new(key, AES.MODE_CBC, IV)
    padding = AES.block_size - len(source) % AES.block_size  # calculate needed padding
    source += bytes([padding]) * padding  # Python 2.x: source += chr(padding) * padding
    data = IV + encryptor.encrypt(source)  # store the IV at the beginning and encrypt
    return base64.b64encode(data).decode("latin-1") if encode else data

def decrypt(key, source, decode=True):
    if decode:
        source = base64.b64decode(source.encode("latin-1"))
    key = SHA256.new(key).digest()  # use SHA-256 over our key to get a proper-sized AES key
    IV = source[:AES.block_size]  # extract the IV from the beginning
    decryptor = AES.new(key, AES.MODE_CBC, IV)
    data = decryptor.decrypt(source[AES.block_size:])  # decrypt
    padding = data[-1]  # pick the padding value from the end; Python 2.x: ord(data[-1])
    if data[-padding:] != bytes([padding]) * padding:  # Python 2.x: chr(padding) * padding
        raise ValueError("Invalid padding...")
    return data[:-padding]  # remove the padding

def encrypt_file(password, file_name):
    e =[]
    r = open(file_name, "r")
    for line in r:
        e.append(encrypt(bytes(password, 'utf-8'), bytes(line,'utf-8'), encode=True))
    r.close()
    w = open(file_name, "w")
    for each in e:
        w.write(each + "\n")
    w.close()

def decrypt_file(password, file_name):
    d =[]
    r = open(file_name, "r")
    for line in r:
        d.append(decrypt(bytes(password, 'utf-8'), line, decode=True))
    r.close()
    w = open(file_name, "w")
    for each in d:
        w.write(each.decode("utf-8"))
    w.close()

In [127]:
encrypt_file(input("Encryption Key: "), input("File Name: "))

Encryption Key:  test
File Name:  EncryptedData.txt


In [128]:
decrypt_file(input("Encryption Key: "), input("File Name: "))

Encryption Key:  test
File Name:  EncryptedData.txt
